# Case study: queue analysis

This case study illustrates the use of the staircase package for queue analysis.  In this example we have a number of vessels (i.e. ships) which arrive offshore and await their turn to enter a harbour where they will be loaded with cargo.  We will examine the queue, which is composed of all vessels which are offshore but yet to enter the harbour, for the year 2020.

The data used is this case study is synthetic and fictional.  Both data and the notebook for this tutorial can be obtained from the [github site](https://github.com/venaturum/staircase/tree/master/docs/examples).

In [ ]:
import pandas as pd
import staircase as sc
import matplotlib.pyplot as plt

We begin by importing the queue data into a pandas.DataFrame instance. Each row corresponds to a vessel. The first column gives the time at which the vessel arrives offshore (enters the queue), and the second column gives the time at which the vessel enters the harbour (leaves the queue).  A [NaT](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html#datetimes) value in either of these columns indicates the vessel entered the queue prior to 2020, or left the queue after 2020, however this approach does not require these values to be NaT.  The third column gives the weight of cargo destined for the vessel.  Note, for the staircase approach to work we require every vessel, that was in the queue at some point in 2020, to appear in the dataframe.

In [ ]:
data = pd.read_csv(r"data/vessel_queue.csv", parse_dates=['enter', 'leave'], dayfirst=True)
data

The layer method can be used with array-like parameters.  The creation of a step function to quantify the size of the queue is as simple as calling the layer method with a vector of times that vessels enter the queue, and a vector of times that vessels leave the queue - the columns "enter" and "leave" respectively:

In [ ]:
queue = sc.Stairs(use_dates=True).layer(data.enter, data.leave)
queue.plot()

Assuming that no vessels arrive precisely at midnight on the 1st of Jan, we expect the number of vessels in the queue at this time to be equal to the number of NaT values in the "enter" column:

In [ ]:
queue(pd.Timestamp('2020-01-01'))

In [ ]:
data.enter.isna().sum()

Another useful queue metric is the "queue tonnes".  This is the sum of the cargo tonnes destined for vessels in the queue.  A step function representing this variable is also straightforward by using the third parameter of the layer method - the values representing how much the step function should increase or decrease whenever the corresponding vessels enter or leave the queue:

In [ ]:
queue_tonnes = sc.Stairs(use_dates=True).layer(data.enter, data.leave, data.tonnes)
queue_tonnes.plot()

We can use this queue_tonnes object to answer questions like "what was the maximum queue tonnes in 2020?"

In [ ]:
queue_tonnes.max()

.. or "what was the average size of the queue in 2020?"

In [ ]:
queue_tonnes.mean(pd.Timestamp('2020'), pd.Timestamp('2021'))

or "what fraction of the year was the queue_tonnes larger than 1,500,000 tonnes?"

In [ ]:
(queue_tonnes > 1500000).mean(pd.Timestamp('2020'), pd.Timestamp('2021'))

.. or "what was the median size of the queue in March 2020?"

In [ ]:
queue_tonnes.median(pd.Timestamp('2020-3-1'), pd.Timestamp('2020-4-1'))

The median gives us the 50th percentile, but we might be interested in the 80th percentile?  We can do that:

In [ ]:
queue_tonnes.percentile(80, pd.Timestamp('2020-3-1'), pd.Timestamp('2020-4-1'))

In fact we can even get a percentile function, represented by a Stairs object itself.  This function is essentially the inverse of an [empirical cumulative distribution function](https://en.wikipedia.org/wiki/Empirical_distribution_function).

In [ ]:
inv_ecdf = queue_tonnes.percentile_Stairs(pd.Timestamp('2020'), pd.Timestamp('2021'))
inv_ecdf

We can plot this function of course, since it is represented by a Stairs object:

In [ ]:
inv_ecdf.plot()

The 100th percentile should be the same as the maximum queue tonnes we found earlier.  Let's check:

In [ ]:
inv_ecdf(100) == queue_tonnes.max()

What is the 40th, 65th, 77th and 90th percentiles?  The sample method, which is aliased by __call__, can be called with a vector of values at which to evaluate the step function too.

In [ ]:
inv_ecdf([40, 65, 77, 90])

Returning to our queue plots... They're pretty noisy.  Perhaps a daily average will suffice.  To achieve this let's use Python's zip function, list comprehension and a pandas.Series to derive and collect this data:

In [ ]:
yr2020 = pd.date_range('2020', '2021')
daily_mean_queue = pd.Series(
    [queue.mean(d1,d2) for d1,d2 in zip(yr2020[:-1], yr2020[1:])],
    index = yr2020[:-1]
)
daily_mean_queue

We can call the [pandas.Series.plot](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.plot.html) method for a quick visualisation.

In [ ]:
daily_mean_queue.plot()

Since the data is now in a series it's easy to apply a rolling window.  This data can be plotted with matplotlib, or seaborn, but for now let's keep leveraging the pandas.Series plotting methods:

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
daily_mean_queue.plot(ax=ax, label="queue size")
daily_mean_queue.rolling(7, center=True).mean().plot(ax=ax, linewidth=3, label="rolling mean")
ax.legend()